In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#from xgboost import plot_importance

from sklearn.model_selection import GridSearchCV#, train_test_split
#from sklearn.metrics import accuracy_score, precision_score, recall_score,\
#f1_score, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay

from sklearn.ensemble import GradientBoostingRegressor,HistGradientBoostingRegressor
import tensorflow as tf

#from sklearn.preprocessing import StandardScaler, normalize

#import re
#import seaborn as sns
import math
#import category_encoders as ce

from scipy.stats import boxcox#, boxcox_normmax, norm, skew
#from scipy.special import boxcox1p
import statistics

from xgboost import XGBRegressor
from sklearn.linear_model import LassoCV
from lightgbm import LGBMRegressor

2024-06-12 03:17:37.130645: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 03:17:37.130809: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 03:17:37.295772: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [3]:
train_house = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_house = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
all_data = pd.concat([train_house, test_house], axis=0)

In [4]:
def eda_fill_missing(df):
    df.loc[df["MSZoning"].isna(),"MSZoning"] = 'RL'
    df.loc[df["Exterior1st"].isna(),"Exterior1st"] = 'VinylSd'
    df.loc[df["Exterior2nd"].isna(),"Exterior2nd"] = 'VinylSd'
    df.loc[df["BsmtFinSF1"].isna(),"BsmtFinSF1"] = df['BsmtFinSF1'].mean()
    df.loc[df["BsmtFinSF2"].isna(),"BsmtFinSF2"] = df['BsmtFinSF2'].mean()
    df.loc[df["BsmtUnfSF"].isna(),"BsmtUnfSF"] = df['BsmtUnfSF'].mean()
    df.loc[df["TotalBsmtSF"].isna(),"TotalBsmtSF"] = df['TotalBsmtSF'].mean()
    df.loc[df["Electrical"].isna(),"Electrical"] = 'SBrkr'
    df.loc[df["BsmtFullBath"].isna(),"BsmtFullBath"] = df['BsmtFullBath'].mean()
    df.loc[df["BsmtHalfBath"].isna(),"BsmtHalfBath"] = 0.0
    df.loc[df["Functional"].isna(),"Functional"] = 'Typ'
    df.loc[df["GarageCars"].isna(),"GarageCars"] = df['GarageCars'].mean()
    df.loc[df["GarageArea"].isna(),"GarageArea"] = df['GarageArea'].mean()
    df.loc[df["SaleType"].isna(),"SaleType"] = 'WD'
    df.loc[df["Utilities"].isna(),"Utilities"] = 'AllPub'
    df.loc[df["MasVnrArea"].isna(),"MasVnrArea"] = df['MasVnrArea'].mean()
    df.loc[df["BsmtQual"].isna(),"BsmtQual"] = 'TA'
    df.loc[df["BsmtCond"].isna(),"BsmtCond"] = 'TA'
    df.loc[df["BsmtExposure"].isna(),"BsmtExposure"] = 'No'
    df.loc[df["BsmtFinType1"].isna(),"BsmtFinType1"] = 'Unf'
    df.loc[df["BsmtFinType2"].isna(),"BsmtFinType2"] = 'Unf'
    df.loc[df["GarageType"].isna(),"GarageType"] = 'Attchd'
    return df

In [5]:
def eda_drop_missing(df):
    df=df.drop(['Alley','PoolQC','MiscFeature','MasVnrType','FireplaceQu'], axis=1)
    return df #,'Fence'

In [6]:
def eda_ave_missing(df):
    df.loc[df['GarageYrBlt'].isna(),"GarageYrBlt"]=0
    df['GarageYrBlt']=df['GarageYrBlt'].astype('int64')
    df.loc[df["LotFrontage"].isna(),"LotFrontage"] = df['LotFrontage'].mean()
    df.loc[df["GarageType"].isna(),"GarageType"] = 'Attchd'
    df.loc[df["GarageYrBlt"].isna(),"GarageYrBlt"] = round(df['GarageYrBlt'].mean())
    df.loc[df["GarageFinish"].isna(),"GarageFinish"] = 'Unf'
    df.loc[df["GarageQual"].isna(),"GarageQual"] = 'TA'
    df.loc[df["GarageCond"].isna(),"GarageCond"] = 'TA'
    return df

In [7]:
def convert_cat(df):
    df[["MSSubClass","YearBuilt","YearRemodAdd","GarageYrBlt","YrSold"]] = df[["MSSubClass","YearBuilt","YearRemodAdd","GarageYrBlt","YrSold"]].astype("category")
    return df

In [8]:
def unskew(df):
    skewed = [
    'LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
    'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea',
    'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
    'ScreenPorch', 'PoolArea', 'LowQualFinSF', 'MiscVal',
    'KitchenAbvGr','TotRmsAbvGrd','Fireplaces'
    ]

    for i in skewed:
        adder=abs(df[i].mean())
        df_temp, fitted = boxcox(np.array(df[i]-df[i].min()+adder))#-df[i].min()+adder
        df[i]=df_temp
    return df

In [9]:
def conv_MoSold(df):
    df["MoSold"]=df["MoSold"].astype("category")
    return df

In [10]:
def feature_eng(df):
    #Areas
    df["TotalArea"]  = df["GrLivArea"] + df["TotalBsmtSF"]
    df["TotalBaths"] = df["FullBath"] + df["BsmtFullBath"] + 0.5*(df["HalfBath"] + df["BsmtHalfBath"])
    df["TotalPorch"] = df["OpenPorchSF"] + df["EnclosedPorch"] + df["3SsnPorch"] + df["ScreenPorch"]
    
    #Logical features
    df['Pool'] = df['PoolArea'].apply(lambda x: True if x > 0 else False).astype("bool")
    df['2ndFloor'] = df['2ndFlrSF'].apply(lambda x: True if x > 0 else False).astype("bool")
    df['Garage'] = df['GarageCars'].apply(lambda x: True if x > 0 else False).astype("bool")
    df['Bsmt'] = df['TotalBsmtSF'].apply(lambda x: True if x > 0 else False).astype("bool")
    df['Fireplace'] = df['Fireplaces'].apply(lambda x: True if x > 0 else False).astype("bool")
    df['Porch'] = df['TotalPorch'].apply(lambda x: True if x > 0 else False).astype("bool")
    
    return df

In [11]:
def eda_correction(df):
    df['GarageYrBlt']=df['GarageYrBlt'].astype('int')
    df.iloc[2592,59]=2007
    df['GarageYrBlt']=df['GarageYrBlt'].astype('category')
    df = df[(df["GrLivArea"]<4450) | (df["Id"]==2550)] #6/2
    df = df[(df["TotalBsmtSF"]<5000) | (df["Id"]==2550)] #6/2
    return df

In [12]:
def change_year(df):
    df['YearBuilt']=df['YearBuilt'].astype('int')
    df['YearBuilt']=df['YearBuilt']-1875
    df['YearRemodAdd']=df['YearRemodAdd'].astype('int')
    df['YearRemodAdd']=df['YearRemodAdd']-1950
    df['GarageYrBlt']=df['GarageYrBlt'].astype('int')
    df['GarageYrBlt']=df['GarageYrBlt']-1895
    return df

In [13]:
# Reduce Neighbor into 4, 
def simplify_neighbor(df):
    df['Neighbor2'] = 2
    df['Neighbor2']=df['Neighbor2'].astype('int')
    L=90

    for i in range(df.shape[0]):
        if (df.iloc[i,12]=="Edwards")|(df.iloc[i,12]=="IDOTRR")|(df.iloc[i,12]=="MeadowV")|(df.iloc[i,12]=="OldTown")|(df.iloc[i,12]=="Sawyer"):# 12=Neighbor
            df.iloc[i,L]=1
        elif (df.iloc[i,12]=="BrkSide")|(df.iloc[i,12]=="Somerst")|(df.iloc[i,12]=="StoneBr")|(df.iloc[i,12]=="Timber"):
            df.iloc[i,L]=3
        elif (df.iloc[i,12]=="NoRidge")|(df.iloc[i,12]=="NridgHt"):
            df.iloc[i,L]=4
    return df

In [14]:
# Reduce GrLivArea into 5, 
def simplify_grlivarea(df): #46=GrLivArea
    df['GrLivArea2']=np.round(df['GrLivArea']/1000).astype('int')
    return df

In [15]:
# Grouping , Neighbor2 + OverallQual
def grouping(df):
    df['Grouping'] = df['Neighbor2'].astype('str') + "_" + df['OverallQual'].astype('str') #6/10 2354 GrLivArea2削除してみた

    return df

In [16]:
df01=eda_fill_missing(all_data)
df02=eda_ave_missing(df01)
df03=convert_cat(df02)
df04=unskew(df03)
df05=conv_MoSold(df04)
df06=feature_eng(df05)
df07=eda_correction(df06)
df08=change_year(df07)
df09=simplify_neighbor(df08)
df10=simplify_grlivarea(df09)

df12 = grouping(df10)

In [17]:
# Generate Dummy
inp = df12
df13 = pd.get_dummies(inp, drop_first=True, 
                      columns=['MSSubClass','MSZoning','Street','LotShape','LandContour','LotConfig',
                               'LandSlope','Condition1','BldgType','HouseStyle',#'Neighborhood',
                               'RoofStyle','RoofMatl','Exterior1st','ExterQual',
                               'Foundation','BsmtQual','BsmtExposure',
                               'HeatingQC','CentralAir','KitchenQual',
                               'FireplaceQu','GarageType','GarageFinish','GarageQual',
                               'PavedDrive','SaleType','SaleCondition','MoSold',"Grouping",
                               "Alley", "PoolQC", "MiscFeature",
                               'MasVnrType'
                               ]) 
df14 = df13.drop(['Id','Exterior2nd','Utilities','Condition2','ExterCond',
                  'BsmtCond','BsmtFinType1','BsmtFinType2','Heating','Electrical','Functional','GarageCond',"YrSold",
                  'LowQualFinSF','BsmtHalfBath','3SsnPorch','MiscVal','Neighborhood',
                  'BsmtFinSF1','BsmtFinSF2','1stFlrSF','2ndFlrSF','Fence'
                  
                  
                 ], axis=1)

In [18]:
# Prepare Train & Test data w/ Survived
df22=df14[df14['SalePrice'].notna()] #テストデータ含まず
y=df22['SalePrice']
X=df22.drop(['SalePrice'], axis=1)
X_train = X
y_train = np.round(y)

In [19]:
# 1) XGBoost regressor
xgb = XGBRegressor(objective='reg:squarederror', random_state=0)#reg:linear, reg:squarederror

# Create a dictionary of hyperparameters to tune
cv_params = {'max_depth': [2,3,4,5], #best [2,5,7,9][4,5][2,3,4,5]
             'min_child_weight': [2],#best [3,5,7,9]
             'learning_rate': [0.05,0.1],#0.01,0.02,0.1
             'n_estimators': [390,400,410] #best 20,40,60,80,,[250,260,270,280]
             # best 0.89565 = 0.1, 4, 280
             }

# Instantiate the GridSearchCV object
xgb_cv = GridSearchCV(xgb, cv_params, cv=4)


In [20]:
%%time
xgb_cv.fit(X_train, y_train)
print(xgb_cv.best_score_)
print(xgb_cv.best_params_)

0.8917029989534844
{'learning_rate': 0.05, 'max_depth': 4, 'min_child_weight': 2, 'n_estimators': 410}
CPU times: user 6min 43s, sys: 2.07 s, total: 6min 45s
Wall time: 1min 49s


In [21]:
# 3) Gradient Boosting Regressor
gbr = GradientBoostingRegressor(loss='huber',random_state=0)#squarederror, # try max_features='sqrt'

# Create a dictionary of hyperparameters to tune
cv_params = {'max_depth': [2,3,4,5],# best [2,3,4,5]
             'learning_rate': [0.1],
             'n_estimators': [280,290,300]  #best [60,80,100], 6/3 original 250,260,270, 300,310,320
             }

# Instantiate the GridSearchCV object
gbr_cv = GridSearchCV(gbr, cv_params, cv=4)

In [22]:
%%time
gbr_cv.fit(X_train, y_train)
print(gbr_cv.best_score_)
print(gbr_cv.best_params_)

0.8950143849261519
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 290}
CPU times: user 2min 59s, sys: 31.2 ms, total: 2min 59s
Wall time: 2min 59s


In [23]:
#5 HGBR : HistGradientBoostingRegressor 
hgbr = HistGradientBoostingRegressor(loss='poisson',random_state=0)#squarederror, absolute_error', 'poisson', 'quantile'

# Create a dictionary of hyperparameters to tune
cv_params = {'max_depth': [3,4,5],#best [2,3,4,5]
             'learning_rate': [0.05],
             'max_iter': [380,400,420] #[50,100,150][380,400,420][410,420,430]
             }

# Instantiate the GridSearchCV object
hgbr_cv = GridSearchCV(hgbr, cv_params, cv=4)# refit='accuracy' scoring=scoring,

In [24]:
%%time
hgbr_cv.fit(X_train, y_train)
print(hgbr_cv.best_score_)
print(hgbr_cv.best_params_)

0.9012663984936297
{'learning_rate': 0.05, 'max_depth': 4, 'max_iter': 400}
CPU times: user 3min 8s, sys: 4.33 s, total: 3min 12s
Wall time: 49.6 s


In [25]:
# Lasso
lasso_cv = LassoCV(random_state=0, cv=20, alphas=[0.01,0.1,1,10,100,1000,10000,100000]).fit(X_train, y_train) #cv20=0.7332

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4396733752.883301, tolerance: 877590187.5232123
  model = cd_fast.enet_coordinate_descent_gram(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9535484150.954102, tolerance: 877590187.5232123
  model = cd_fast.enet_coordinate_descent_gram(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 165459442945.36353, tolerance: 877590187.5232123
  model = cd_fast.enet_coordinate_descent_gram(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWa

In [26]:
print(lasso_cv.score(X_train, y_train)) #alpha=100, 0.88669

0.8806573052490758


In [27]:
# LGBMR
lgbm = LGBMRegressor(num_leaves=4,n_estimators=5000,max_bin=200,bagging_fraction=0.75,
                     bagging_freq=5,bagging_seed=7,feature_fraction=0.2,
                     feature_fraction_seed=7,verbose=-1)

# Create a dictionary of hyperparameters to tune
cv_params = {'learning_rate': [0.01,0.05,0.1],
             'n_estimators': [5000]
             }

# Instantiate the GridSearchCV object
lgbm_cv = GridSearchCV(lgbm, cv_params, cv=4)# cv=4, refit='accuracy' scoring=scoring,       

In [28]:
%%time
lgbm_cv.fit(X_train, y_train)
print(lgbm_cv.best_score_)
print(lgbm_cv.best_params_)

0.8916032293791287
{'learning_rate': 0.01, 'n_estimators': 5000}
CPU times: user 28.4 s, sys: 1.42 s, total: 29.8 s
Wall time: 29.2 s


In [29]:
# RUN
X_test_final = df14[df14['SalePrice'].isna()].drop(['SalePrice'], axis=1) #テストデータのみ

In [30]:
# Ensemble 2, voting
def ensamble2(X, m1, m2, m3, m4, m5):
    y_pred1 = m1.best_estimator_.predict(X) # XGB
    y_pred2 = m2.best_estimator_.predict(X) # HGBR
    y_pred3 = m3.best_estimator_.predict(X) # GBR
    y_pred4 = m4.predict(X) # LassoCV
    y_pred5 = m5.best_estimator_.predict(X) #LGBM
    y_pred  = (y_pred1 + y_pred2 + y_pred3 + y_pred4 + y_pred5) / 5 *0.998 # Ensemble
    y_pred = np.array(y_pred, dtype='int')    
    return y_pred
 
y_pred_final = ensamble2(X_test_final, xgb_cv,hgbr_cv ,gbr_cv, lasso_cv, lgbm_cv)

In [31]:
submission = pd.DataFrame({
        "Id": test_house["Id"],
        "SalePrice": y_pred_final
    })
submission.to_csv('./temp1.csv', index=False)